In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Download training data from open datasets. 
train_data = datasets.MNIST(
    root="data",
    train=True, 
    download=True,
    transform=ToTensor()
    )


# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False, 
    download=True,
    transform=ToTensor()
    )

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [51]:
# Hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 50

In [31]:
# Create data loaders.
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [28]:
for X, y in test_dataloader:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape}")
    break

Shape of X: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])


In [52]:
import torch.nn.functional as F

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


# ~~ Model ~~
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        input = 28*28
        hidden = 512
        output = 10

        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(input, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, output)
        )

    # Forward Pass
    def forward(self, x):
        x = self.flatten(x)
        out = self.layers(x)
        return out


model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [32]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [46]:
# Training Loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss = loss.item()
            current = batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


# Test Loop
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [50]:
# Epochs
try:
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer)
        test_loop(test_dataloader, model, loss_fn)
    print("Done!")
    torch.save(model.state_dict(), "model.pth")
    print("Saved Model State to model.pth")
except KeyboardInterrupt:
    # Save model training progress. 
    torch.save(model.state_dict(), "model.pth")
    print("Saved Model State to model.pth")

Epoch 1
-------------------------------
loss: 2.245683  [    0/60000]
loss: 2.248158  [ 6400/60000]
loss: 2.247702  [12800/60000]
loss: 2.248788  [19200/60000]
loss: 2.248897  [25600/60000]
loss: 2.251332  [32000/60000]
loss: 2.237065  [38400/60000]
loss: 2.253375  [44800/60000]
loss: 2.238450  [51200/60000]
loss: 2.262363  [57600/60000]
Test Error: 
 Accuracy: 32.9%, Avg loss: 2.226839 

Epoch 2
-------------------------------
loss: 2.217195  [    0/60000]
loss: 2.229361  [ 6400/60000]
loss: 2.202537  [12800/60000]
loss: 2.218936  [19200/60000]
loss: 2.207987  [25600/60000]
loss: 2.225086  [32000/60000]
loss: 2.225024  [38400/60000]
loss: 2.208313  [44800/60000]
loss: 2.184345  [51200/60000]
loss: 2.174157  [57600/60000]
Test Error: 
 Accuracy: 39.1%, Avg loss: 2.184050 

Epoch 3
-------------------------------
loss: 2.205744  [    0/60000]
loss: 2.198678  [ 6400/60000]
loss: 2.169452  [12800/60000]
loss: 2.145407  [19200/60000]
loss: 2.148461  [25600/60000]
loss: 2.176127  [32000/600

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Image'